In [1]:
from db_utils import *
from config import generate_config_json

import dragonfly #DO NOT REINSTALL--THE SITE-PACKAGES VERSION OF DRAGONFLY IS FIXED/MODIFIED
from dragonfly import load_config_file
from dragonfly.exd.experiment_caller import CPFunctionCaller
from dragonfly.opt import gp_bandit

/home/ubuntu/.local/lib/python3.10/site-packages/dragonfly/utils/oper_utils.py:30: UserWarning: cannot import name 'direct' from 'dragonfly.utils.direct_fortran' (/home/ubuntu/.local/lib/python3.10/site-packages/dragonfly/utils/direct_fortran/__init__.py)
Could not import Fortran direct library. Dragonfly can still be used, but might be slightly slower. To get rid of this warning, install a numpy compatible Fortran compiler (e.g. gfortran) and the python-dev package and reinstall Dragonfly.
  warn('%s\n%s'%(e, fortran_err_msg))


In [3]:
#LOWER BOUND CANNOT BE MORE THAN 0
ALL_CONSIDERED_COMPONENTS = [
    ('Ca(ClO4)2', (0,7.5)),
    ('LiPF6', (0,7.5)),
    ('CaCl2', (0,7.5)),
    ('H6C4O3',(0,7.5))
]
EXP_COMPONENTS = [
    'CaCl2',
    'H6C4O3',
    'LiPF6'
]
# in mL
MAX_VOL = 100.0
MIN_VOL = 2.0

EXPERIMENTER_INITIALS = 'AC'
CONFIG_OUTPUT_FILE = './config'
#OR
CONFIG_INPUT_FILE = ''

In [4]:

all_considered_components_formatted = []
for chem, bounds in ALL_CONSIDERED_COMPONENTS:
    all_considered_components_formatted.append((get_component_type(chem),bounds))

exp_components_formatted = [get_component_type(chem) for chem in EXP_COMPONENTS]
exp_config_path, constraint_path = generate_config_json(
    all_considered_components_formatted,
    exp_components_formatted,
    MAX_VOL,
    MIN_VOL,
    EXPERIMENTER_INITIALS,
    CONFIG_OUTPUT_FILE
                                       )
print(exp_config_path + ': New config file generated')

./config/exp_2023-08-14T22-44-25.json: New config file generated


In [5]:
config = load_config_file(exp_config_path)
#optimization method is bayesian optimization
domain, domain_orderings = config.domain, config.domain_orderings
func_caller = CPFunctionCaller(None, domain, domain_orderings=domain_orderings)
opt = gp_bandit.CPGPBandit(func_caller, ask_tell_mode=True)
opt.initialise()

/home/ubuntu/.local/lib/python3.10/site-packages/dragonfly/exd/exd_core.py:338: UserWarning: Sampling an initial pool failed despite 10 attempts -- will continue trying but consider reparametrising your domain if this problem persists.
  warn(('Sampling an initial pool failed despite %d attempts -- will ' +
/home/ubuntu/.local/lib/python3.10/site-packages/dragonfly/exd/exd_core.py:338: UserWarning: Sampling an initial pool failed despite 20 attempts -- will continue trying but consider reparametrising your domain if this problem persists.
  warn(('Sampling an initial pool failed despite %d attempts -- will ' +
/home/ubuntu/.local/lib/python3.10/site-packages/dragonfly/exd/exd_core.py:338: UserWarning: Sampling an initial pool failed despite 30 attempts -- will continue trying but consider reparametrising your domain if this problem persists.
  warn(('Sampling an initial pool failed despite %d attempts -- will ' +
/home/ubuntu/.local/lib/python3.10/site-packages/dragonfly/exd/exd_core.p

In [24]:
experiment_electrolytes = get_experiment_electrolytes([tupl[0] for tupl in ALL_CONSIDERED_COMPONENTS])
opt.tell([(point['amounts'], point['electrolyte'][1]) for point in experiment_electrolytes])
#TELL ALL EXISTING DATABASE POINTS

In [27]:
opt.ask()

[0.0, 2.477, 4.73, 2.012]